# Quantum Development Kit Samples<br>Diagnostics: Facts and Assertions

## Preamble

We use the Microsoft.Quantum.Standard.Visualization package to provide additional visualization support for the diagnostics in this notebook.

In [1]:
%package Microsoft.Quantum.Standard.Visualization

Adding package Microsoft.Quantum.Standard.Visualization: done!

Microsoft.Quantum.Standard::0.12.20082513, Microsoft.Quantum.Standard.Visualization::0.12.20082513

For convinence, we'll also open the [Microsoft.Quantum.Diagnostics namespace](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics) so that its functions and operations are available throughout the notebook.

In [2]:
open Microsoft.Quantum.Diagnostics;

## Facts: Conditions on values

The Q# standard libraries provide several different ways to validate the correctness of quantum programs as functions and operations in the [Microsoft.Quantum.Diagnostics namespace](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics).
The first and most basic of these are *facts*, which represent an invariant that always holds for a given value.

In [3]:
function AdditionFact() : Unit {
    Fact(2 + 3 == 5, "addition fact failed");
}

AdditionFact

In [4]:
%simulate AdditionFact

()

From this example, you can see that when a fact is true, nothing happens. On the other hand, if the condition for a fact fails, then the Q# program is terminated with the message associated with the fact. Try modifying the example above and see what happens when you run `%simulate AdditionFact` again.

One important thing to note is that facts depend only on *values*. That is, `2 + 3 == 5` is a value of type `Bool` that is either `true` or `false` irrespective of the state of your target machine, or of any qubits in your device. As a consequence, facts can be expressed as functions that are easy to use either in other functions, or in adjointable and controllable operations. For example, let's take a quick look at how the [`Fact` function](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics.fact) is defined in the Q# standard libraries:

```qsharp
function Fact(actual : Bool, message : String) : Unit {
    if (not actual) { fail message; }
}
```

Similarly, the [`Microsoft.Quantum.Diagnostics.Contradiction` function](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics.contradiction) function can be used to ensure that a value is definitely `false` at some point in a Q# program:

In [5]:
function AdditionContradiction() : Unit {
    Contradiction(2 + 3 == 6, "addition contradiction failed");
}

AdditionContradiction

In [6]:
%simulate AdditionContradiction

()

In practice, it can be helpful to test conditions that have more structure than a `Bool` value, such as that two values are equal to each other. This allows for printing out helpful information when things go wrong; let's try that out by updating our definition of `AdditionFact` to something incorrect and see what happens:

In [7]:
function AdditionFact() : Unit {
    // This should fail, since it is decidedly not a fact that 2 + 3 is equal to 6.
    EqualityFactI(2 + 3, 6, "addition fact failed");
}

AdditionFact

In [8]:
%simulate AdditionFact

Microsoft.Quantum.IQSharp.DisplayableException

addition fact failed
	Expected:	6
	Actual:	5


One particularly important example of when facts can be useful is in comparing the results we get from measuring a register of qubits to what we would expect to get based on the state we tried to prepare. For instance, consider the following operation to prepare a pair of entangled qubits (itself a special case of the [`PrepareEntangledState` operation](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.preparation.prepareentangledstate)):

In [9]:
operation PrepareEntangledPair(left : Qubit, right : Qubit) : Unit is Adj + Ctl {
    H(left);
    CNOT(left, right);
}

PrepareEntangledPair

We can check that this operation works the way would expect by checking some properties of its output state. In particular, we expect that this operation will prepare the state $\left(\left|00\right\rangle + \left|11\right\rangle\right) / \sqrt{2}$ when given two qubits initially in the $\left|00\right\rangle$ state. Thus, measuring the parity of the prepared state should always return `Zero`:

In [10]:
operation CheckEntanglementPreparation() : Unit {
    using ((left, right) = (Qubit(), Qubit())) {
        PrepareEntangledPair(left, right);
        let actual = Measure([PauliZ, PauliZ], [left, right]);
        EqualityFactR(actual, Zero, "Parity of (|00⟩ + |11⟩) / √2 was not Zero.");
    }
}

CheckEntanglementPreparation

In [11]:
%simulate CheckEntanglementPreparation

()

For more examples, check out the list of fact functions in the [Microsoft.Quantum.Diagnostics namespace](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics) documentation.

## Assertions: Conditions on machine state

By comparison, we sometimes want to check a condition on the state of our machine or of some qubits within a device. Unlike facts, conditions of this form can't be represented as functions, since a condition on a `Qubit` value may be true or false depending on the side effects accumulated in a program so far — not just depending on the `Qubit` itself.

Thus, conditions on states need to represented as *operations* rather than as *functions*. Let's take a look at how we can use operations to validate that freshly allocated qubits start off in the $\left|0\right\rangle$ state:

In [12]:
operation CheckFreshQubitIsInZero() : Unit {
    using (q = Qubit()) {
        AssertMeasurement([PauliZ], [q], Zero, "qubit was not in |0⟩");
    }
}

CheckFreshQubitIsInZero

In [13]:
%simulate CheckFreshQubitIsInZero

()

When run on a simulator that supports it, [`AssertMeasurement`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics.assertmeasurement) checks that **if** you were to measure `[q]` in the `[PauliZ]` basis, then it you would get the result `Zero` with **certainty**. Since the only state which always returns a measurement result of `Zero` when measured in the `PauliZ` basis is the $\left|0\right\rangle$ state, passing the assertion tells us that our qubit is definitely in the $\left|0\right\rangle$ state immediately afterwards.

> ⚠ **NOTE**: This assertion critically depends on being able to violate the no-cloning theorem, since it checks a hypothetical measurement without actually performing that measurement.
> That said, since the assertion has no effect on the state of the target machine, the assertion will automatically be ignored when run on simulators which don't support it, or on actual hardware that is bound by the no-cloning theorem.
> This allows us to safely use assertions to check and validate assumptions when we can, while still allowing our Q# programs to work correctly across target machines.

The same assertion can be used to check other conditions as well; for example, $H\left|0\right\rangle = \left|+\right\rangle$, so we would expect that if we apply the `H` operation to a fresh qubit, then our qubit will be in the $+1$ eigenstate of the $X$-basis.

In [14]:
operation CheckHPreparesPlus() : Unit {
    using (q = Qubit()) {
        within { H(q); }
        apply {
            AssertMeasurement([PauliX], [q], Zero, "qubit was not in |+⟩");
        }
    }
}

CheckHPreparesPlus

In [15]:
%simulate CheckHPreparesPlus

()

The Q# standard libraries also provide additional assertions built on top of [`AssertMeasurement`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics.assertmeasurement) and [`AssertMeasurementProbability`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics.assertmeasurementprobability) that can be convienent for working with different special cases and applications.

For example, [`AssertOperationsEqualReferenced`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics.assertoperationsequalreferenced) makes it easy to check whether two operations have the same effect on a register of qubits. We can use this operation to identities and simpliciations, such as that $(H \otimes H) \mathrm{CNOT}_{01} (H \otimes H) = \mathrm{CNOT}_{10}$:

In [16]:
operation ApplyCnotDirectly(left : Qubit, right : Qubit) : Unit is Adj + Ctl {
    CNOT(left, right);
}

operation ApplyCnotWithinH(left : Qubit, right : Qubit) : Unit is Adj + Ctl {
    within {
        ApplyToEachCA(H, [left, right]);
    } apply {
        CNOT(right, left);
    }
}

ApplyCnotDirectly, ApplyCnotWithinH

In [17]:
operation CheckCnotIdentity() : Unit {
    AssertOperationsEqualReferenced(2,
        ApplyToFirstTwoQubitsCA(ApplyCnotDirectly, _),
        ApplyToFirstTwoQubitsCA(ApplyCnotWithinH, _)
    );
}

CheckCnotIdentity

In [18]:
%simulate CheckCnotIdentity

()

Similarly, the Microsoft.Quantum.Diagnostics namespace provides useful shorthands for commonly used assertions. For example, [`AssertQubit`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics.assertqubit) makes it easy to assert measurement outcomes on a single qubit in the $Z$-basis.

In [19]:
operation CheckFreshQubitIsInZero() : Unit {
    using (q = Qubit()) {
        AssertQubit(Zero, q);
    }
}

CheckFreshQubitIsInZero

In [20]:
%simulate CheckFreshQubitIsInZero

()

## Allowances: Conditions on control flow

The last kind of condition supported by the Q# standard libraries checks control flow within a segment of a Q# program. For example, if you want to make sure that your Q# program never allocates more than a given number of qubits, that can be checked using the [`AllowAtMostNQubits` operation](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics.allowatmostnqubits):

In [21]:
operation CheckNoLargeRegistersAllocated() : Unit {
    within {
        AllowAtMostNQubits(6, "More than six qubits were allocated");
    } apply {
        using (register = Qubit[7]) {
            // ...
        }
    }
}

CheckNoLargeRegistersAllocated

In [22]:
%simulate CheckNoLargeRegistersAllocated

Source,Callable
(notebook),CheckNoLargeRegistersAllocated


7 qubit(s) were allocated, but at most 6 qubit(s) are allowed:
More than six qubits were allocated.


Similarly, the [`AllowAtMostNCallsCA` operation](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.diagnostics.allowatmostncallsca) can be used to ensure that a given operation is not called more than an allowed number of times:

In [23]:
operation CheckHNotCalled() : Unit {
    within {
        AllowAtMostNCallsCA(0, H, "The H operation was called.");
    } apply {
        using (q = Qubit()) {
            H(q);
            H(q);
        }
    }
}

CheckHNotCalled

In [24]:
%simulate CheckHNotCalled

Calls to Microsoft.Quantum.Intrinsic.H:\n\n- Microsoft.Quantum.Intrinsic.H

Unhandled exception. Microsoft.Quantum.Simulation.Core.ExecutionFailException: Operation Microsoft.Quantum.Intrinsic.H was called more than the allowed 0 times:
The H operation was called.
 ---> Microsoft.Quantum.Intrinsic.H on :line 0
   at SNIPPET.CheckHNotCalled on C:\snippet_.qs:line 0


Operation Microsoft.Quantum.Intrinsic.H was called more than the allowed 0 times:
The H operation was called.


## Epilogue

In [25]:
%version

Component,Version
iqsharp,0.12.20082513
Jupyter Core,1.4.0.0
.NET Runtime,".NETCoreApp,Version=v3.1"
